In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
os.listdir('./')

['.ipynb_checkpoints',
 'gender_submission.csv',
 'submission.csv',
 'test.csv',
 'titanic_analysis01.ipynb',
 'titanic_analysis02.ipynb',
 'titanic_analysis03.ipynb',
 'train.csv']

In [3]:
df = pd.read_csv('./train.csv')
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
df_cabin = df[["Pclass","Ticket","Fare","Embarked", "Cabin"]]
df_cabin = df_cabin.dropna(how="any").reset_index(drop=True)
df_cabin

,Pclass,Ticket,Fare,Embarked,Cabin
0,1,PC 17599,71.2833,C,C85
1,1,113803,53.1000,S,C123
2,1,17463,51.8625,S,E46
3,3,PP 9549,16.7000,S,G6
4,1,113783,26.5500,S,C103
...,...,...,...,...,...
197,1,11751,52.5542,S,D35
198,1,695,5.0000,S,B51 B53 B55
199,1,11767,83.1583,C,C50
200,1,112053,30.0000,S,B42


In [32]:
tempList = []
def filterOutlier(x) :
    for i in x :
        if(i.find(' ')>0 or len(i) == 1):
            tempList.append(np.nan)
        else:
            tempList.append(i)
    return tempList

In [33]:
df_cabin["valid"] = df_cabin[["Cabin"]].apply(lambda x: filterOutlier(x))

In [34]:
df_cabin = df_cabin.dropna(how = "any").reset_index(drop = True)
df_cabin

,Pclass,Ticket,Fare,Embarked,Cabin,valid
0,1,PC 17599,71.2833,C,C85,C85
1,1,113803,53.1000,S,C123,C123
2,1,17463,51.8625,S,E46,E46
3,3,PP 9549,16.7000,S,G6,G6
4,1,113783,26.5500,S,C103,C103
...,...,...,...,...,...,...
169,1,PC 17590,50.4958,S,A24,A24
170,1,11751,52.5542,S,D35,D35
171,1,11767,83.1583,C,C50,C50
172,1,112053,30.0000,S,B42,B42


In [39]:
df_cabin["Cabin"].unique()
list = []
for i in df_cabin["Cabin"]:
    list.append(i[0])
ser = pd.Series(list)
print(ser.unique())

['C' 'E' 'G' 'D' 'A' 'B' 'F']


In [76]:
tempList=[]
def seperateFloor(x):
    for i in x:
        if pd.isna(i):
            tempList.append(np.nan)
        elif i[0] == 'A':
            tempList.append(7)
        elif i[0] == 'B':
            tempList.append(6)
        elif i[0] == 'C':
            tempList.append(5)
        elif i[0] == 'D':
            tempList.append(4)
        elif i[0] == 'E':
            tempList.append(3)
        elif i[0] == 'F':
            tempList.append(2)
        elif i[0] == 'G':
            tempList.append(1)
        else:
            tempList.append(np.nan)
    return tempList

In [77]:
df_cabin["Floor"] = df_cabin[["Cabin"]].apply(lambda x: seperateFloor(x))
df_cabin

,Pclass,Ticket,Fare,Embarked,Cabin,valid,Floor
0,1,PC 17599,71.2833,C,C85,C85,5
1,1,113803,53.1000,S,C123,C123,5
2,1,17463,51.8625,S,E46,E46,3
3,3,PP 9549,16.7000,S,G6,G6,1
4,1,113783,26.5500,S,C103,C103,5
...,...,...,...,...,...,...,...
169,1,PC 17590,50.4958,S,A24,A24,7
170,1,11751,52.5542,S,D35,D35,4
171,1,11767,83.1583,C,C50,C50,5
172,1,112053,30.0000,S,B42,B42,6


In [53]:
from sklearn.linear_model import LinearRegression

In [55]:
lr_model = LinearRegression().fit(df_cabin[["Pclass", "Fare"]].join(pd.get_dummies(df_cabin[["Embarked"]])), df_cabin[["Floor"]])

In [56]:
df_x = df[["PassengerId", "Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Embarked", "Cabin"]];
df_y = df[["PassengerId", "Survived"]];
df_x

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Cabin
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,NaN
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C85
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,NaN
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,C123
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S,NaN
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S,B42
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,S,NaN
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C,C148


In [62]:
df_x["Cabin"].unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

In [5]:
# Pclass: A proxy for socio-economic status (SES) 1st = Upper 2nd = Middle 3rd = Lower
# Age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5
# SibSp: The dataset defines family relations in this way…
# Sibling = brother, sister, stepbrother, stepsister
# Spouse = husband, wife (mistresses and fiancés were ignored)
# Parch: The dataset defines family relations in this way…
# Parent = mother, father
# Child = daughter, son, stepdaughter, stepson
# Some children travelled only with a nanny, therefore parch=0 for them.

In [78]:
df_x["Floor"] = df_x[["Cabin"]].apply(lambda x: seperateFloor(x))
df_x

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Cabin,Floor
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,NaN,5.0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C85,5.0
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,NaN,3.0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,C123,1.0
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S,NaN,5.0
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S,B42,NaN
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,S,NaN,NaN
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C,C148,2.0


In [6]:
df_x = df_x.dropna(how = 'any')
df_x

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...
885,886,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,Q
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


In [7]:
df_x = df_x.merge(df_y, left_on="PassengerId", right_on = "PassengerId")
df_x

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Survived
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,1
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,1
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,0
...,...,...,...,...,...,...,...,...,...,...,...
707,886,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,Q,0
708,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S,0
709,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S,1
710,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C,1


In [8]:
from sklearn.linear_model import LinearRegression

In [9]:
df_x = df_x.join(pd.get_dummies(df_x[["Sex"]]))
df_x

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Survived,Sex_female,Sex_male
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,0,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,1,1,0
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,1,1,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,1,1,0
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,886,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,Q,0,1,0
708,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S,0,0,1
709,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S,1,1,0
710,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C,1,0,1


In [10]:
df_y = df_x[["Survived"]]
df_x = df_x[["Pclass","Age","SibSp","Parch","Fare"]].join(pd.get_dummies(df_x[["Sex","Embarked"]]))

In [11]:
lr_model = LinearRegression().fit(df_x, df_y)

In [12]:
df_test_x = pd.read_csv("./test.csv")
df_test_x = df_test_x[["PassengerId","Pclass","Age","SibSp","Parch","Fare"]].join(pd.get_dummies(df_test_x[["Sex","Embarked"]]))
df_test_x

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,0,1,0,1,0
1,893,3,47.0,1,0,7.0000,1,0,0,0,1
2,894,2,62.0,0,0,9.6875,0,1,0,1,0
3,895,3,27.0,0,0,8.6625,0,1,0,0,1
4,896,3,22.0,1,1,12.2875,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,NaN,0,0,8.0500,0,1,0,0,1
414,1306,1,39.0,0,0,108.9000,1,0,1,0,0
415,1307,3,38.5,0,0,7.2500,0,1,0,0,1
416,1308,3,NaN,0,0,8.0500,0,1,0,0,1


In [13]:
df_test_x = df_test_x.fillna(df_test_x.mean())
df_test_y = df_test_x[["PassengerId"]]

In [14]:
result = lr_model.predict(df_test_x.drop("PassengerId",axis=1))

In [15]:
result

array([[ 0.03661737],
       [ 0.42310089],
       [ 0.04780154],
       [ 0.11729809],
       [ 0.57400381],
       [ 0.20089839],
       [ 0.55086184],
       [ 0.25364116],
       [ 0.72657106],
       [ 0.05725156],
       [ 0.0961302 ],
       [ 0.37353997],
       [ 0.96685927],
       [ 0.02611901],
       [ 0.80858876],
       [ 0.8286796 ],
       [ 0.22172586],
       [ 0.22192001],
       [ 0.55172841],
       [ 0.5531679 ],
       [ 0.3378489 ],
       [ 0.22113155],
       [ 0.96104377],
       [ 0.59671237],
       [ 0.87590807],
       [-0.08007892],
       [ 1.07579099],
       [ 0.21228655],
       [ 0.40653375],
       [ 0.06358561],
       [ 0.10960902],
       [ 0.22685895],
       [ 0.49429179],
       [ 0.51237171],
       [ 0.49808312],
       [ 0.23797661],
       [ 0.58154372],
       [ 0.64121524],
       [ 0.13030691],
       [ 0.10565841],
       [ 0.09684094],
       [ 0.47465408],
       [ 0.02722648],
       [ 0.77169817],
       [ 0.81974324],
       [ 0

In [16]:
def classification(x) :
    if x > 0.5 :
        return 1
    else :
        return 0

In [17]:
result = pd.DataFrame(result, columns = ["Survived"])

In [18]:
result["Survived"] = result["Survived"].apply(lambda x: classification(x))

In [19]:
result = result.join(df_test_y)
result = result[["PassengerId","Survived"]]
result

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [20]:
result.to_csv('./submission.csv',index=False)